In [ ]:
import pandas as pd
import cv2
import numpy as np
from skimage.feature import hog


In [ ]:
# Đọc file CSV
data = pd.read_csv('data/data_info.csv', header=None)

images = []
labels = []

# Duyệt qua từng dòng trong file CSV
for idx, row in data.iterrows():
    image_path = row[0]
    num_hands = row[1]
    image = cv2.imread(image_path)

    if num_hands == 0:
        # Nếu không có tay trong ảnh
        images.append(image)
        labels.append(0)
    else:
        for i in range(num_hands):
            x_min, y_min, x_max, y_max = row[2 + i*4 : 6 + i*4]
            hand_region = image[y_min:y_max, x_min:x_max]
            images.append(hand_region)
            labels.append(1)
            
            # Lấy thêm các vùng không chứa tay để cân bằng dữ liệu
            non_hand_region = image[y_max:y_max + (y_max - y_min), x_min:x_max] if y_max + (y_max - y_min) < image.shape[0] else None
            if non_hand_region is not None:
                images.append(non_hand_region)
                labels.append(0)

images = np.array(images)
labels = np.array(labels)


In [ ]:
# Trích xuất HOG cho tất cả ảnh
features = []
for image in images:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
    features.append(feature)

features = np.array(features)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))